### feature합치기
#### investors, historical(매일 거래량 등) and 분류항목 합치기

In [1]:
import datetime, time
from datetime import date

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

In [ ]:
# index column : date
def find_previous_date(df_inv, df_his, date_current):
    i = 0
    date2 = 0
    while True:
        i += 1
        date2 = date_current - datetime.timedelta(days = i)
        if (df_inv.loc[date2]['retail'] != 0) & (df_his.loc[date2]['vol'] != '') :  
            # investors와 historical 두 개 데이터에 같이 없어야 함
            break
        if i >= 30:
            break
    return date2

In [ ]:
def find_ratio(df_o, date_current):
    df_o_trans = df_o.transpose()
    df_o_trans.columns = ['before', 'after']
    df_o_trans[date_current] = (df_o_trans['after'] - df_o_trans['before']) / df_o_trans['before']
    return df_o_trans.transpose()

In [ ]:
def combine_data(df_o, df_inv, df_his):
    investor_rate = pd.DataFrame()
    historical_rate = pd.DataFrame()
    date_weekday = pd.DataFrame()
    for date_current in df_o.index:
        date_previous = find_previous_date(df_inv, df_his, date_current)
        # 거래 전날 요일 구하기
        date_temp = {'date': date_current, 'weekday' : date_previous.weekday()}
        df_temp = pd.DataFrame(date_temp, index=[0]).set_index('date')
        date_weekday = pd.concat([date_weekday, df_temp], axis=0)
        date_previous_1 = find_previous_date(df_inv, df_his, date_previous)
        # 거래 전날, 전전날을 확인하고 변화정도 계산하기(find_ratio)
        df_inv_comp = df_inv.loc[[date_previous_1, date_previous]]
        df_his_comp = df_his.loc[[date_previous_1, date_previous]]
        investor_rate = pd.concat([investor_rate, find_ratio(df_inv_comp, date_current).iloc[[-1]]], axis=0)
        historical_rate = pd.concat([historical_rate, find_ratio(df_his_comp, date_current).iloc[[-1]]], axis=0)
    total = pd.concat([investor_rate, historical_rate, date_weekday, df_o ], axis=1)
    return total

In [ ]:
code = {'005930': ['삼성전자', 'sec'], '005380': ['현대차', 'hyunmotor'],
                 '035420': ['NAVER', 'naver'], '033780': ['KT&G', 'ktng']}
# code = {'005930': ['삼성전자', 'sec']}

In [ ]:
pkl_directory = '../data/company_daily_price_pkl'
pkl_directory = './'
pkl_directory = '../data/company_pkl/'

In [ ]:
hist_column = [ 'date', 'open', 'high', 'low', 'close', 'close_cr', 'vol']
hist_column_m = [ 'date', 'open', 'high', 'low', 'close', 'vol']
inv_column = ['date', 'retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension', 'privequity',
              'bank', 'insurance', 'financeetc', 'corporateetc', 'foreigneretc']

In [ ]:
# investor.pkl, historical.pkl. 읽기
for key, val in code.items():
    pkl_name= '{}.pkl'.format(val[1])
    globals()['df_{}'.format(val[1])] = pd.read_pickle('./' + pkl_name)
    globals()['df_{}'.format(val[1])]['date'] = pd.to_datetime(globals()['df_{}'.format(val[1])]['date']) #change to timestamp
    globals()['df_{}'.format(val[1])]['date'] = globals()['df_{}'.format(val[1])]['date'].dt.date  # change to datetime
    pkl_name= '{}_historical.pkl'.format(val[1])
    globals()['df_{}_{}'.format(val[1], 'historical')] = pd.read_pickle(pkl_directory + pkl_name)
    globals()['df_{}_{}'.format(val[1], 'historical')] = globals()['df_{}_{}'.format(val[1], 'historical')][hist_column_m]
    # close_cr 행을 없앰(변동이 없는 경우가 빌생하여 divided zero error 발생)
    globals()['df_{}_{}'.format(val[1], 'historical')]['date'] = globals()['df_{}_{}'.format(val[1], 'historical')]['date'].dt.date # change to datetime
    pkl_name= '{}_investors.pkl'.format(val[1])
    globals()['df_{}_{}'.format(val[1], 'investors')] = pd.read_pickle(pkl_directory + pkl_name)
    globals()['df_{}_{}'.format(val[1], 'investors')]['date'] = globals()['df_{}_{}'.format(val[1], 'investors')]['date'].dt.date # change to datetime
    
    # get combined data (eg. df_sec_combined, df_ktng_combined...)
    globals()['df_{}_{}'.format(val[1], 'combined')] = combine_data(globals()['df_{}'.format(val[1])].set_index('date'),
                            globals()['df_{}_{}'.format(val[1], 'investors')].set_index('date'),
                            globals()['df_{}_{}'.format(val[1], 'historical')].set_index('date')
                            )

In [ ]:
column_selected = ['retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension', 'privequity',
              'open', 'high', 'low', 'close',  'vol', 'weekday', 'value']

In [ ]:
dd = df_ktng_combined[column_selected] # company, common, df_o data combined.
dd

## 여기까지 완료

## 주요 지표 읽기

In [ ]:
common_pkl = ["bok_rate.pkl", "cpi.pkl", "dji.pkl", "dji_future.pkl", "dxy_future.pkl", 
              "fear_greed.pkl", "fed_rate.pkl", "gold.pkl", "ixic_future.pkl", "kor_10yr_bond.pkl",
              "kor_2yr_bond.pkl", "kosdaq.pkl", "kospi.pkl", "krw_rate.pkl", "nas.pkl",
              "snp_future.pkl", "sox.pkl", "spx.pkl", "us_10yr_bond.pkl", "us_2yr_bond.pkl",
              "us_3mon_bond.pkl", "vix.pkl", "wti_future.pkl"]

In [ ]:
major_index = ["dji.pkl", "nas.pkl", "spx.pkl", "kosdaq.pkl", "kospi.pkl", "sox.pkl", "fear_greed.pkl"]
major_future_index = ["dji_future.pkl", "nas_future.pkl", "spx.pkl", "kosdaq.pkl", "kospi.pkl", "sox.pkl", "fear_greed.pkl"]

In [ ]:
common_pkl[0][:-4]

In [ ]:
pkl_common_directory = '../data/common_pkl/'
for index_name in common_pkl:
    key_name = index_name[:-4]
    globals()['df_{}'.format(key_name)] = pd.read_pickle(pkl_common_directory + index_name)
    combine_data(globals()['df_{}'.format(key_name)].set_index('date'),
                            globals()['df_{}_{}'.format(key_name, 'investors')].set_index('date'),
                            globals()['df_{}_{}'.format(key_name, 'historical')].set_index('date')
                            )

In [ ]:
df_kospi

### feature 이름 설명
#### feature에 사용된 column name 설명

In [ ]:
df_feature_name_abbr = pd.read_csv('../data/feature_name_abbr.csv')

In [ ]:
standforK = lambda x : df_feature_name_abbr['Korean'][df_feature_name_abbr['abbr'] == x].values[0]
standforE = lambda x : df_feature_name_abbr['English'][df_feature_name_abbr['abbr'] == x].values[0]
standforexp = lambda x : df_feature_name_abbr['exp1'][df_feature_name_abbr['abbr'] == x].values[0]

In [ ]:
# financial이 무엇인지 알기 위한 명령
c_name = 'financial'
standforK(c_name), standforE(c_name), standforexp(c_name)

In [ ]:
c_name = 'cpi'
standforK(c_name), standforE(c_name), standforexp(c_name)

### 설명에서 feature name 찾기

In [ ]:
find_col_nameK = lambda x : df_feature_name_abbr[['abbr', 'Korean']][df_feature_name_abbr['Korean'].str.contains(x)]
find_col_nameE = lambda x : df_feature_name_abbr[['abbr', 'English']][df_feature_name_abbr['English'].str.contains(x, na=False, case=False)]

In [ ]:
e_word = 'INDEX' # English column에서 찾기
find_col_nameE(e_word)

In [ ]:
k_word = '반도체' # Korean column에서 찾기
find_col_nameK(k_word)

### Read and Combine data 

In [2]:
bokd = '../data/common_pkl/bok_rate.pkl'
cpid = '../data/common_pkl/cpi.pkl'
djid = '../data/common_pkl/dji.pkl'
bok = pd.read_pickle(bokd)
cpi = pd.read_pickle(cpid)
dji = pd.read_pickle(djid)
boks = pd.read_pickle(bokd).set_index('date')
cpis = pd.read_pickle(cpid).set_index('date')
djis = pd.read_pickle(djid).set_index('date')

In [3]:
end_date = datetime.datetime.today().date()
start_date = '20220620'
date_range_ts = pd.date_range(start=start_date, end=end_date)

In [4]:
df_base = pd.DataFrame(pd.Series(date_range_ts, name='date')).set_index('date')

In [5]:
df_base = df_base.merge(cpis, how='left', left_index=True, right_index=True)

In [6]:
df_base = df_base.merge(djis, how='left', left_index=True, right_index=True)

In [7]:
df_base.head()

,cpi,cpi_anticipated,cpi_previous,dji,open,high,low,volume,dji_cr
date,,,,,,,,,
2022-06-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-21,NaN,NaN,NaN,30531.77,30074.69,30653.71,30074.69,345.14M,+2.16%
2022-06-22,NaN,NaN,NaN,30481.88,30352.57,30777.92,30174.02,327.81M,-0.16%
2022-06-23,NaN,NaN,NaN,30678.15,30570.33,30715.63,30294.37,347.93M,+0.64%
2022-06-24,NaN,NaN,NaN,31503.71,30881.77,31513.93,30881.77,398.11M,+2.69%


In [8]:
df_base.tail()

,cpi,cpi_anticipated,cpi_previous,dji,open,high,low,volume,dji_cr
date,,,,,,,,,
2023-02-15,NaN,NaN,NaN,34128.34,34008.63,34128.34,33834.7,260.62M,+0.11%
2023-02-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
